# Numerical Methods in Accelerator Physics

### Demo lecture: phase-space tomography

<br />

##### Notebook lecture by [Dr. Adrian Oeftiger $\nearrow$](http://web-docs.gsi.de/~aoeftige)

at TU Darmstadt etit, Fachbereichsrats-Sitzung, on 27.09.2022.

<br />
<center>
    <small>(hit space to go to the next slide)</small>
</center>

#### Run this notebook online:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/aoeftiger/TUDa-demo-lecture-tomo/v1.1?labpath=lecture.ipynb)

Also find this lecture rendered [on github $\nearrow$](https://aoeftiger.github.io/TUDa-demo-lecture-tomo/) along with the [source repository $\nearrow$](https://github.com/aoeftiger/TUDa-demo-lecture-tomo/):

https://github.com/aoeftiger/TUDa-demo-lecture-tomo

# Tomographic Reconstruction

<b>"Tomography"</b>: imaging via sectioning

<b>Origins</b>: mathematician J. Radon (AUT)
- 1917: "On the determination of functions from their integrals along certain manifolds"
- inverse problem
- <b>Fourier slice theorem</b>: any 2D (3D) object can be reconstructed from infinite set of 1D (2D) projections

<div style="float:right; width:15%;">
    <a title="daveynin from United States, CC BY 2.0 &lt;https://creativecommons.org/licenses/by/2.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:UPMCEast_CTscan.jpg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/UPMCEast_CTscan.jpg/800px-UPMCEast_CTscan.jpg" alt="CT scanner" style="width:100%;" /></a>
    <a title="Arielinson, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:3d_CT_scan_animation.gif"><img src="https://upload.wikimedia.org/wikipedia/commons/6/68/3d_CT_scan_animation.gif" alt="Reconstructed CT scan" style="width:100%;" /></a>
    <img src="img/tomo.png" alt="Phase-space tomography at CERN PSB" style="width:100%;" />
<!--<a title="LucasVB, CC0, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Radon_transform_sinogram.gif"><img alt="Radon transform sinogram" src="https://upload.wikimedia.org/wikipedia/commons/9/93/Radon_transform_sinogram.gif" style="width:100%;" /></a>-->
</div>

# Many applications

- medical: CT scan in hospitals (computed tomography)
    - 1979 Nobel price in Medicine: <br /> first CT scanner by Sir G.N. Hounsfield
- material science
- airport security
- accelerator physics
- ...

# Projection Integral or Radon Transform $\mathcal{R}_\theta(p)$

$$\require{color}
\mathcal{R}_\theta(p)\, f = \int dx\int dy~ f(x, y) \,\underbrace{\delta(x\,\cos \theta+y\,\sin\theta-p)}\limits_{\color{red}\text{projection slice}}$$

<img src="img/radon.png" alt="Radon transform" style="width:30%;margin:auto;" />

<p style="font-size: small; text-align: right;">image source: <a href="https://twitter.com/docmilanfar/status/1366229985975930881/photo/1">@docmilanfar, Twitter</a></p>

In [ ]:
# imports
from talksetup import *
%matplotlib inline

from skimage.transform import radon, iradon
from PIL import Image

Load sample image for the tomography:

In [ ]:
data = ~np.array(Image.open('etit_logo.png').convert('1', dither=False))

plt.imshow(data, cmap='binary')

Compute the Radon transform at an angle of 0 deg and 30 deg:

In [ ]:
Rf_0 = radon(data, [0], circle=False).astype(float)
Rf_90 = radon(data, [90], circle=False).astype(float)

plt.plot(Rf_0, label='0 deg')
plt.plot(Rf_90, label='90 deg')
plt.legend()
plt.xlabel('$p$')
plt.ylabel(r'$\mathcal{R}_{\theta}(p)f$');

# Reconstruction Principle

<img src="img/iter1.gif" alt="projection" style="width:20%;float:left;margin-right:5%;margin-left:2.5%;margin-bottom:1em;margin-top:1em;" />
<img src="img/iter2.gif" alt="back-projection" style="width:20%;float:left;margin-right:5%;margin-bottom:1em;margin-top:1em;" />

<p style="clear: both; font-size: small; text-align: right; margin-top:1em;">images source: <a href="https://tomograp.web.cern.ch/home_page/iter_tomo/iter_tomo/">CERN tomography website</a></p>

projection (measurement) $\implies$ back-projection (reconstruction) 

(Require $ N_{meas} \gtrsim \pi\cdot\frac{\text{total diameter}}{\text{pixel size}}$)

# Reconstruction Principle

<img src="img/iter1.gif" alt="projection" style="width:20%;float:left;margin-right:5%;margin-left:2.5%;margin-bottom:1em;margin-top:1em;" />
<img src="img/iter2.gif" alt="back-projection" style="width:20%;float:left;margin-right:5%;margin-bottom:1em;margin-top:1em;" />
<img src="img/iter3.gif" alt="back-projection" style="width:20%;float:left;margin-right:5%;margin-bottom:1em;margin-top:1em;" />
<img src="img/iter4.gif" alt="back-projection" style="width:20%;float:left;margin-bottom:1em;margin-top:1em;" />

<p style="clear: both; font-size: small; text-align: right; margin-top:1em;">images source: <a href="https://tomograp.web.cern.ch/home_page/iter_tomo/iter_tomo/">CERN tomography website</a></p>

projection (measurement) $\implies$ back-projection (reconstruction) 

$\stackrel{\text{improve}}{\implies}$ re-projection (red) $\implies$ iteratively reduce discrepancy

# Reconstruction Algorithms

<div style="float:right;width:30%;">
<a title="Biomedizinische NMR Forschungs GmbH, CC BY-SA 3.0 &lt;https://creativecommons.org/licenses/by-sa/3.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Heart-direct-vs-iterative-reconstruction.png"><img src="https://upload.wikimedia.org/wikipedia/commons/5/54/Heart-direct-vs-iterative-reconstruction.png" alt="direct vs. iterative reconstruction" style="width:100%;" /></a>
</div>

<b>Filtered Back-Projection (FBP)</b> vs. <b>Iterative</b> Reconstruction

$\implies$ Iterative <b>algebraic reconstruction technique (ART)</b>:
- more computationally expensive than FBP
- more accurate, less artifacts
- can incorporate <i>a priori</i> knowledge

<!-- $$ \sum\limits_j A_{ij} x_j = b_i $$
$$ x^{k+1} = x^k + \frac{b_i - a_i\cdot x^k}{|a_i|^2} a_i^T $$
-->

In [ ]:
# parameters
N = max(data.shape)
ANG = 180
VIEW = 180
THETA = np.linspace(0, ANG, VIEW, endpoint=False)

In [ ]:
# definitions of matrix transforms
A = lambda x: radon(x, THETA, circle=False).astype(float)
AT = lambda y: iradon(y, THETA, circle=False, filter=None, 
                      output_size=N).astype(float) * 2 * len(THETA) / np.pi
AINV = lambda y: iradon(y, THETA, circle=False, output_size=N).astype(float)

In [ ]:
proj = A(data)

In [ ]:
plt.imshow(proj.T)

plt.gca().set_aspect('auto')
plt.xlabel('Projection location [px]')
plt.ylabel('Rotation angle [deg]')
plt.colorbar(label='Intensity');

In [ ]:
fbp = AINV(proj)

In [ ]:
plt.imshow(fbp, cmap='binary', vmin=0, vmax=1);

In [ ]:
noise = np.random.normal(0, 0.2 * np.max(proj), size=proj.shape)

In [ ]:
proj_noise = proj + noise

In [ ]:
plt.imshow(proj_noise.T)
plt.gca().set_aspect('auto')
plt.xlabel('Projection location [px]')
plt.ylabel('Rotation angle [deg]')
plt.colorbar(label='Intensity');

In [ ]:
fbp_noise = AINV(proj_noise)

In [ ]:
plt.imshow(fbp_noise, cmap='binary', vmin=0, vmax=1, interpolation='None');

In [ ]:
def ART(A, AT, b, x, mu=1, niter=10):
    ATA = AT(A(np.ones_like(x)))

    for i in tqdm(range(niter)):
        x = x + np.divide(mu * AT(b - A(x)), ATA)

        # nonlinearity: constrain to >= 0 values
        x[x < 0] = 0

        plt.imshow(x, cmap='binary', vmin=0, vmax=1, interpolation='None')
        plt.title("%d / %d" % (i + 1, niter))
        plt.show()

    return x

# initialisation
x0 = np.zeros((N, N))
mu = 1
niter = 10

In [ ]:
x_art = ART(A, AT, proj_noise, x0, mu, niter)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 5))
plt.subplots_adjust(wspace=0.3)

plt.sca(ax[0])
plt.imshow(fbp_noise, cmap='binary', vmin=0, vmax=1, interpolation='None')
plt.title('FBP')

plt.sca(ax[1])
plt.imshow(x_art, cmap='binary', vmin=0, vmax=1, interpolation='None')
plt.title('ART');

# Tomography in Accelerator Physics

Longitudinal phase space reconstruction:

<div style="float: left; width:23%;">
<img src="img/distr_ini.png" style="width:100%;" />
<img src="img/waterfall.png" style="width:100%;" />
<img src="img/distr_fin2.png" style="width:100%;" />
</div>
<img src="img/tomo_sim_out.png" style="float: left; width:43%; margin-left:10%;" />

# Summary

- Radon transform
- Tomographic reconstruction algorithms:
    - filtered back-projection
    - iterative: algebraic reconstruction technique

<br />

$\implies$ Go ahead, try out the <a href="./acc-tomo.ipynb">accelerator physics example $\nearrow$</a>

Further literature:
[Chapter 3 of A. C. Kak and Malcolm Slaney, Principles of Computerized Tomographic Imaging, Society of Industrial and Applied Mathematics, 2001](https://engineering.purdue.edu/~malcolm/pct/CTI_Ch03.pdf)